In [1]:
!pip install -q requests beautifulsoup4 google-generativeai


In [12]:
import google.generativeai as genai

genai.configure(api_key="YOUR_GEMINI_API_KEY")

model = genai.GenerativeModel("gemini-2.5-flash")


In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def fetch_page(url):
    try:
        response = requests.get(url, timeout=15)
        soup = BeautifulSoup(response.text, "html.parser")

        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()

        text = soup.get_text(separator=" ")
        text = " ".join(text.split())

        return soup, text
    except:
        return None, ""


In [4]:
def extract_internal_links(soup, base_url):
    links = set()
    base_domain = urlparse(base_url).netloc

    for a in soup.find_all("a", href=True):
        href = urljoin(base_url, a["href"])
        if urlparse(href).netloc == base_domain:
            links.add(href.split("#")[0])

    return list(links)


In [5]:
def planner_agent(question):
    prompt = f"""
You are a planning agent.

Goal: Answer the question using website evidence.

Question: {question}

Create a step-by-step plan for how a web agent should navigate and verify.
Only output steps. Do not answer.
"""
    return model.generate_content(prompt).text


In [6]:
def evidence_agent(content, question):
    prompt = f"""
You are an evidence-checking agent.

Question: {question}

Website content:
\"\"\"
{content[:3000]}
\"\"\"

Does this content contain strong evidence to answer the question?

Reply EXACTLY in this format:
RESULT: YES or NO
REASON: one short line
"""
    return model.generate_content(prompt).text


In [7]:
def decision_agent(evidence, question):
    prompt = f"""
You are a reasoning agent.

Answer the question using ONLY the evidence below.
If evidence is insufficient, say NOT FOUND.

Question: {question}

Evidence:
\"\"\"
{evidence}
\"\"\"
"""
    return model.generate_content(prompt).text


In [8]:
def crawling_web_agent(start_url, question, max_pages=6):
    visited = set()
    to_visit = [start_url]
    evidence_memory = []

    print("🤖 AGENT ACTIVATED")
    print("🎯 GOAL:", question)

    print("\n🧠 PLANNING...")
    print(planner_agent(question))

    while to_visit and len(visited) < max_pages:
        current_url = to_visit.pop(0)

        if current_url in visited:
            continue

        print(f"\n🌐 VISITING PAGE: {current_url}")
        visited.add(current_url)

        soup, text = fetch_page(current_url)
        if not soup:
            print("⚠️ Failed to read page, skipping")
            continue

        print("🔍 ANALYZING CONTENT...")
        evaluation = evidence_agent(text, question)
        print(evaluation)

        if "RESULT: YES" in evaluation:
            print("✅ EVIDENCE FOUND!")
            evidence_memory.append(text)
            break
        else:
            print("❌ Not found here. Deciding next action...")

        print("🔗 DISCOVERING NEW LINKS...")
        links = extract_internal_links(soup, current_url)

        for link in links:
            if link not in visited and link not in to_visit:
                to_visit.append(link)

        print(f"📌 Pages queued: {len(to_visit)}")

    print("\n🧠 FINAL DECISION...")
    if evidence_memory:
        return decision_agent("\n".join(evidence_memory), question)
    else:
        return "NOT FOUND: The agent explored the website but could not verify the answer."


In [11]:
url = "https://www.geeksforgeeks.org/"
question = "Does this website provide machine learning courses? if yes then provide the exact link"

answer = crawling_web_agent(url, question)
print("\n🟢 FINAL ANSWER:\n", answer)


🤖 AGENT ACTIVATED
🎯 GOAL: Does this website provide machine learning courses? if yes then provide the exact link

🧠 PLANNING...
1.  Navigate to the website's homepage.
2.  Scan the main navigation menu (header, sidebar) and footer for links commonly associated with educational offerings, such as "Courses," "Programs," "Academics," "Learning," "Degrees," "Certifications," "Training," "Study," or "Curriculum."
3.  If a promising navigation link is found, click on it. On the subsequent page(s), search for keywords like "machine learning," "ML," "artificial intelligence," "AI," "deep learning," "neural networks," or "data science."
4.  If no obvious navigation links are found or the search on those pages yields no results, locate the website's search bar (if available).
5.  Use the search bar with queries such as "machine learning courses," "ML courses," "artificial intelligence courses," "AI training," or "data science programs."
6.  Examine the search results. If any results appear relev